In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from io import BytesIO

np.random.seed(0)

In [37]:
def add_constant(ctx, array):
    print(type(array))
    return pc.add(array, 1, memory_pool=ctx.memory_pool)

In [90]:
def area(ctx,array):
    format = pc.list_element(array, 4)
    result = []
    for i in range(40):
        if i == 0 :
            mask = pc.equal(format,i)
            result.append(pc.if_else(mask,pc.multiply(pc.list_element(array,2),pc.list_element(array,3)),None))
        elif i == 1 :
            mask = pc.equal(format,i)
            result.append(pc.if_else(mask,pc.multiply(pc.subtract(pc.list_element(array,3),pc.list_element(array,1)),pc.subtract(pc.list_element(array,2),pc.list_element(array,0))),None))

    final_result = 0
    for i, item in enumerate(result):
        if i+1 < len(result):
            final_result = item.fill_null(result[i+1])
    
    return final_result

In [39]:
#table = pq.read_table(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\imagenette\imagenette2\output\ds.parquet")
table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")

In [105]:
test_array = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("bbox")
#test_array = table.column("image_feature").chunk(0).values.field("class_feature").values.field("label")
type(test_array[0][0])

pyarrow.lib.DoubleScalar

In [82]:
format = pc.list_element(test_array, 4)
result = []
for i in range(40):
    if i == 0 :
        mask = pc.equal(format,i)
        result.append(pc.if_else(mask,pc.multiply(pc.list_element(test_array,2),pc.list_element(test_array,3)),None))
    elif i == 1 :
        mask = pc.equal(format,i)
        result.append(pc.if_else(mask,pc.multiply(pc.subtract(pc.list_element(test_array,3),pc.list_element(test_array,1)),pc.subtract(pc.list_element(test_array,2),pc.list_element(test_array,0))),None))

final_result = 0
for i, item in enumerate(result):
    if i+1 < len(result):
        final_result = item.fill_null(result[i+1])

In [88]:
final_result = 0
for i, item in enumerate(result):
    if i+1 < len(result):
        final_result = item.fill_null(result[i+1])

In [89]:
final_result

[
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  ...
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999
]

In [109]:
import pyarrow as pa
import pyarrow.compute as pc


func_doc = {}
func_doc["summary"] = "simple bbox"
func_doc["description"] = "calculate bbox area"
func_name = "area"
in_types = {"array": pa.list_(pa.float64())}
out_type = pa.float64()

pc.register_scalar_function(area, func_name, func_doc, in_types, out_type)
func = pc.get_function(func_name)

answer = pc.call_function(func_name, [test_array])


In [110]:
answer

[
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  ...
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999
]